# OCI Vision API – Image Analysis (Beginner Notebook)

This step-by-step notebook lets you analyze images using Oracle Cloud Infrastructure Vision services. You'll upload an image to Object Storage and let OCI Vision automatically detect objects, text, faces, and classify it – all with the official SDK.

Based on `vision/oci_vision.py` script.

## 1. Setup and Requirements

- Your *sandbox.yaml* and *.env* should be set up with relevant bucket/OCI credentials
- Install required packages (run this in a terminal):
```bash
pip install oci python-dotenv envyaml
```
- Place your target image in the location you want (default is `vision/receipt.png` but you can upload any image!)

In [1]:
import os
from dotenv import load_dotenv
from envyaml import EnvYAML
from pathlib import Path
import oci
from oci.object_storage import ObjectStorageClient
load_dotenv()

## 2. Load OCI/Sandbox Configuration

In [2]:
SANDBOX_CONFIG_FILE = "sandbox.yaml"
DEFAULT_FILE = Path("./vision/receipt.png")

def load_config(config_path):
    try:
        return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None
    except Exception as e:
        print(f"Error loading config: {e}")
        return None

scfg = load_config(SANDBOX_CONFIG_FILE)
assert scfg is not None and 'oci' in scfg and 'bucket' in scfg, "Check your sandbox.yaml config!"
oci_cfg = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]), scfg["oci"]["profile"])
bucket_cfg = scfg["bucket"]
compartment_id = scfg["oci"]["compartment"]

## 3. Select and Display an Image

In [3]:
from IPython.display import Image, display
# Change this if you want to use another image:
FILE_PATH = Path("vision/receipt.png")
# Example: FILE_PATH = Path("vision/dussera-b.jpg")

display(Image(filename=str(FILE_PATH)))

## 4. Upload Image to OCI Object Storage

In [4]:
def upload(oci_cfg, bucket_cfg, file_path):
    if not file_path.exists():
        print(f"Error: File '{file_path}' not found.")
        return False
    object_storage_client = ObjectStorageClient(oci_cfg)
    print(f"Uploading file {file_path} ...")
    object_storage_client.put_object(
        bucket_cfg["namespace"], 
        bucket_cfg["bucketName"], 
        f"{bucket_cfg['prefix']}/{file_path.name}", 
        open(file_path, 'rb')
    )
    print("Upload completed!")
    return True

uploaded = upload(oci_cfg, bucket_cfg, FILE_PATH)
if not uploaded:
    raise ValueError("Upload failed. Check your image and configuration.")

## 5. Set Up Analysis Features

In [5]:
def get_image_location(bucket_cfg, file_name):
    image = oci.ai_vision.models.ObjectStorageImageDetails(
        source="OBJECT_STORAGE",
        namespace_name=bucket_cfg["namespace"],
        bucket_name=bucket_cfg["bucketName"],
        object_name=f"{bucket_cfg['prefix']}/{file_name}"
    )
    return image

def get_analyze_image_details(features, compartment_id, bucket_cfg, file_name):
    details = oci.ai_vision.models.AnalyzeImageDetails(
        features=features,
        image=get_image_location(bucket_cfg, file_name),
        compartment_id=compartment_id
    )
    return details

features = [
    oci.ai_vision.models.ImageClassificationFeature(),
    oci.ai_vision.models.ImageObjectDetectionFeature(),
    oci.ai_vision.models.ImageTextDetectionFeature(),
    oci.ai_vision.models.FaceDetectionFeature()
]

## 6. Analyze Image!

In [6]:
vision_client = oci.ai_vision.AIServiceVisionClient(config=oci_cfg)
try:
    response = vision_client.analyze_image(
        get_analyze_image_details(features, compartment_id, bucket_cfg, FILE_PATH.name)
    )
    print("Analysis complete! (Status: ", response.status, ")")
except Exception as e:
    print(f"Error during analysis: {e}")
    response = None

## 7. Parse and Print Results

In [7]:
def parse_vision_response(data):
    print("\nOCI Vision Analysis Results:")
    labels = getattr(data, 'labels', None)
    if labels:
        print("\nClassifications:")
        for label in labels:
            print(f"  - {label.name}: {label.confidence:.2f}")
    image_objects = getattr(data, 'image_objects', None)
    if image_objects:
        print("\nDetected Objects:")
        for obj in image_objects:
            print(f"  - {obj.name} : {obj.confidence:.2f}")
    image_text = getattr(data, 'image_text', None)
    if image_text and hasattr(image_text, 'lines'):
        print("\nDetected Text Lines:")
        for line in image_text.lines:
            print(f"  - {line.text}: {line.confidence:.2f}")
    detected_faces = getattr(data, 'detected_faces', None)
    if detected_faces:
        print("\nDetected Faces:")
        for face in detected_faces:
            print(f"  - Face confidence: {face.confidence:.2f} : Quality {face.quality_score:.2f}")
    if not any([labels, image_objects, image_text, detected_faces]):
        print("No features detected in the image.")

if response is not None and hasattr(response, 'data'):
    parse_vision_response(response.data)


## 🎉 Done! Next Steps

- Try different images or text files for analysis.
- Adapt feature selection for narrower use-cases (object-detection only, for instance).
- Explore downstream automation: parse response dicts for structured results!

If you have OCI API errors, check your config, permissions, and credentials.